In [1]:
import ee
import folium
import geehydro
import geemap

C:\Users\srini\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
ee.Initialize(project='ml-meets-the-sun')

In [3]:
Map = geemap.Map(center = (15.3173,75.7139), zoom =5)
Map.add_basemap('HYBRID')

In [4]:
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [5]:
karnataka = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Karnataka')
TamilNadu = ee.FeatureCollection('projects/ml-meets-the-sun/assets/TamilNadu')
Rajasthan = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Rajasthan')

In [6]:
def map_display(state,name):
    vis_params = {
    'color': 'red'
    }
    Map.addLayer(state, vis_params, name, True, 0.5)
    return Map
Map

Map(bottom=7610.0, center=[19.01668213079985, 75.97045898437501], controls=(WidgetControl(options=['position',…

In [7]:
states = {
    'Karnataka': karnataka,
    'Tamil Nadu': TamilNadu,
    'Rajasthan': Rajasthan
}
for name,state in states.items():
    map_display(state,name)

In [8]:
dataset = ee.ImageCollection('ESA/WorldCover/v200').first()
visualization = {
  'bands': ['Map'],
};

Map.centerObject(dataset);

Map.addLayer(dataset, visualization, 'Landcover');

In [9]:
dataset.projection()

In [12]:
result_clip = {}

def lulc_clip(state, name):
    cname = name + "_lulc"
    clipped = dataset.clip(state)
    result_clip[cname] = clipped

In [13]:
for name,state in states.items():
   lulc_clip(state, name)

In [14]:
result_clip

{'Karnataka_lulc': <ee.image.Image at 0x17d0baff5b0>,
 'Tamil Nadu_lulc': <ee.image.Image at 0x17d0baff790>,
 'Rajasthan_lulc': <ee.image.Image at 0x17d0bafed40>}

In [17]:
def display_clipped_lulc(cname, clipped):
    
    visualization = {
    'bands': ['Map'],
     };
   
    Map.addLayer(clipped,visualization, cname, True, 0.5)
    return Map

In [18]:
for cname,clipped in result_clip.items():
    display_clipped_lulc(cname, clipped)

In [22]:
result_clip['Karnataka_lulc'].projection()

In [30]:
region = karnataka.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Karnataka_lulc'],
    description='Karnataka_lulc',
    assetId='projects/ml-meets-the-sun/assets/KA_lulc',
    region=region,
    crsTransform = [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [31]:
region = TamilNadu.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Tamil Nadu_lulc'],
    description='Tamil Nadu_lulc',
    assetId='projects/ml-meets-the-sun/assets/TN_lulc',
    region=region,
    crsTransform = [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [32]:
region = Rajasthan.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Rajasthan_lulc'],
    description='Rajasthan_lulc',
    assetId='projects/ml-meets-the-sun/assets/RA_lulc',
    region=region,
    crsTransform = [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()